In [4]:
require 'hdf5'
require 'torch'
require 'cutorch'
require 'cunn'
require 'optim'
require 'xlua'
require 'paths'
require 'unsup'

c = torch.load('master_experiments/2016-01-17-13-50/workers/worker-96.608231707317.t7')

-- define target classes
classes = {'WALKING', 'WALKING_UPSTAIRS', 'WALKING_DOWNSTAIRS', 'SITTING',
'STANDING', 'LAYING'}
-- this matrix records the current confusion across classes
confusion = optim.ConfusionMatrix(classes)
confusion:zero()

-- load validation and test data
myFile = hdf5.open("data.hdf5", 'r')
train_x = myFile:read('train_x'):all()
train_y = myFile:read('train_y'):all():cuda()
val_x = myFile:read('val_x'):all()
val_y = myFile:read('val_y'):all():cuda()
test_x = myFile:read('test_x'):all()
-- zca whitening for train data, 
-- also store mean, and p matrix for validation and test data transformation
train_x, zca_mean, zca_p, zca_invp = unsup.zca_whiten(train_x)
val_x = unsup.zca_whiten(val_x, zca_mean, zca_p, zca_invp)
test_x = unsup.zca_whiten(test_x, zca_mean, zca_p, zca_invp)
train_x = train_x:cuda()
val_x = val_x:cuda()
test_x = test_x:cuda()
myFile:close()
collectgarbage()

-- load models for ensembling

-- Code by David Kastrup
require "lfs"

function dirtree(dir)
  assert(dir and dir ~= "", "directory parameter is missing or empty")
  if string.sub(dir, -1) == "/" then
    dir=string.sub(dir, 1, -2)
  end

  local function yieldtree(dir)
    for entry in lfs.dir(dir) do
      if entry ~= "." and entry ~= ".." then
        entry=dir.."/"..entry
	local attr=lfs.attributes(entry)
	coroutine.yield(entry,attr)
	if attr.mode == "directory" then
	  yieldtree(entry)
	end
      end
    end
  end

  return coroutine.wrap(function() yieldtree(dir) end)
end

worker_list = {}
i = 1
for filename in dirtree("./master_experiments/2016-01-17-14-13/workers") do
    worker_list[i] = filename
    i = i + 1
end
table.sort(worker_list)

models = {}
for i = 1, 7 do
    models[i] = torch.load(worker_list[#worker_list - i + 1])['model']:cuda()
end
--[[models = {
    --torch.load("master_experiments/2016-01-17-14-13/workers/worker-97.005208333333.t7")['model']:cuda(),
    --torch.load("master_experiments/2016-01-17-14-13/workers/worker-96.653846153846.t7")['model']:cuda(),
    torch.load("master_experiments/2016-01-17-14-13/workers/worker-96.056547619048.t7")['model']:cuda(),
    --torch.load("master_experiments/2016-01-17-14-13/workers/worker-95.807926829268.t7")['model']:cuda(),
    --torch.load("master_experiments/2016-01-17-14-13/workers/worker-95.667613636364.t7")['model']:cuda(),
    --torch.load("master_experiments/2016-01-17-14-13/workers/worker-95.525568181818.t7")['model']:cuda(),
    --torch.load("master_experiments/2016-01-17-14-13/workers/worker-95.46130952381.t7")['model']:cuda(),
    --torch.load("master_experiments/2016-01-17-14-13/workers/worker-92.48046875.t7")['model']:cuda(),
    }--]]

function val()
    preds = torch.Tensor(val_x:size(1), #classes):zero()
    for m = 1, #models do
        models[m]:evaluate()
        for i=1, val_x:size(1) do
            pred = models[m]:forward(val_x[i]):double()
            preds[i] = preds[i] + pred
        end
    end
    for i=1, preds:size(1) do
        confusion:add(preds[i], val_y[i])
    end
    
    print(confusion)
    val_accuracy = confusion.totalValid * 100
    -- reset confusion matrix
    confusion:zero()
    print('val_accuracy: ' .. val_accuracy .. '%')

    return val_accuracy
end

function ensemble_predict()
    preds = torch.FloatTensor(test_x:size(1),#classes):zero()
    
    for m = 1, #models do
        models[m]:evaluate()
        for i=1, test_x:size(1) do
            pred = models[m]:forward(test_x[i]):float()
            preds[i] = preds[i] + pred
        end
    end
    local f = torch.DiskFile('ensemble_results/result-ensemble-' .. ensemble_val .. '.txt', 'w')
    for i=1, preds:size(1) do
        max_v, max_i = torch.max(preds[i], 1)
        f:writeFloat(max_i[1])
    end
    f:close()
end


ensemble_val = val()

ensemble_predict()

ConfusionMatrix:
[[     455      11       0       0       0       0]   97.639% 	[class: WALKING]
 [       2     407       0       0       0       0]   99.511% 	[class: WALKING_UPSTAIRS]
 [       3      12     353       0       0       0]   95.924% 	[class: WALKING_DOWNSTAIRS]
 [       0       0       0     389      26       0]   93.735% 	[class: SITTING]
 [       0       0       0       1     475       0]   99.790% 	[class: STANDING]
 [       0       0       0       3       0     463]]  99.356% 	[class: LAYING]
 + average row correct: 97.659246126811% 
 + average rowUcol correct (VOC measure): 95.592945814133% 
 + global correct: 97.769230769231%
{
  valids : FloatTensor - size: 6
  mat : LongTensor - size: 6x6
  averageUnionValid : 0.95592945814133
  _targ_idx : LongTensor - empty
  averageValid : 0.97659246126811
  classes : 
    {
      1 : WALKING
      2 : WALKING_UPSTAIRS
      3 : WALKING_DOWNSTAIRS
      4 : SITTING
      5 : STANDING
      6 : LAYING
    }
  _prediction : Floa

- maybe the difference is because we use mini-batch validation previously, and now we are using online validation.

In [ ]:
acc = torch.load("master_experiments/2016-01-17-14-13/workers/worker-97.005208333333.t7")['val_acc']

In [ ]:
require "lfs"

In [ ]:
table.sort(worker_list)

In [ ]:
print(worker_list)